In [1]:
#this is necessary for modernbert at the time of this writing, until future releases of transformers
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-p0xeoq8w
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-p0xeoq8w
  Resolved https://github.com/huggingface/transformers.git to commit 5fa35344755d8d9c29610b57d175efd03776ae9e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10474239 sha256=4824dca9e49cb7cd5c07e05fdfb2e74b332b6298ef93b6229c1960f3e4191962
  Stored in directory: /tmp/pip-ephem-wheel-cache-3m4ijcw8/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [2]:
!pip install gradio torch captum seaborn matplotlib shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:
import captum

import torch
import torch.nn as nn
import torch.nn.functional as F
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

import gradio as gr
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification

from captum.attr import (
    LayerIntegratedGradients,
    visualization as viz,
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from IPython.core.display import HTML
import math

In [2]:
model_path = "scbtm/ModernBERT_wine_quality_reviews_ft"
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [4]:
def forward_with_ids(input_ids, attention_mask):
    # pass them along to your model
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    return outputs.logits

lig = LayerIntegratedGradients(forward_with_ids, model.model.embeddings)

In [17]:

def get_embeddings(model, tokenizer, device, text):

    tokens = tokenizer(text, return_tensors="pt", padding=True)
    tokens = {k: v.to(device) for k, v in tokens.items()}

    return model.model.embeddings(tokens['input_ids'])

def generate_baseline(model, tokenizer, device, text):
    tokens = tokenizer(text, return_tensors="pt")
    baseline_tokens = torch.ones_like(tokens['input_ids']) * tokenizer.pad_token_id
    return model.model.embeddings(baseline_tokens.to(device))

def get_target_class(model, tokenizer, device, text):
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    tokens = {k: v.to(device) for k, v in tokens.items()}

    with torch.no_grad():
        outputs = model(**tokens)
        pred_class = torch.argmax(outputs, dim=1)[0].item()
        return pred_class


def get_prediction_and_confidence(model, tokenizer, device, text):
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    tokens = {k: v.to(device) for k, v in tokens.items()}

    with torch.no_grad():
        outputs = model(**tokens).logits
        probs = F.softmax(outputs, dim=1)

        if len(probs.shape) == 1:
            probs = probs.unsqueeze(0)

        pred_class = torch.argmax(probs, dim=1)[0].item()
        #pred_class = min(pred_class, self.num_classes - 1)
        confidence = probs[0, pred_class].item()

    return pred_class, confidence, probs

def get_explanations_ig(model, tokenizer, device, text):
    # 1) Tokenize
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    input_ids = tokens["input_ids"]
    attention_mask = tokens["attention_mask"]

    # 2) Prepare baseline with same shape as input_ids
    baseline_ids = torch.ones_like(input_ids) * tokenizer.pad_token_id
    # If your model expects an attention mask for the baseline, you might do zeros:
    baseline_mask = torch.zeros_like(attention_mask)
    # or all 1s if you prefer. Depends on how you want to handle baseline.

    # 3) Get predicted class
    pred_class, confidence, probs = get_prediction_and_confidence(
        model, tokenizer, device, text
    )

    # 4) Integrated Gradients
    attributions_ig, delta_ig = lig.attribute(
        inputs=(input_ids, attention_mask),
        baselines=(baseline_ids, baseline_mask),
        target=pred_class,
        return_convergence_delta=True,
        n_steps=50,
    )

    return attributions_ig, delta_ig, pred_class, confidence, probs

def merge_subwords(tokens, attributions):
    """
    Given:
      tokens: list of subword tokens (e.g., ["This", "Ġwine", "Ġis", "Ġfrom", "It", "aly", ",", "ĠWhite", "ĠBl", "end"])
      attributions: np.array or list of float scores, same length as tokens

    Returns:
      merged_tokens, merged_attributions
      where subwords have been combined back into words.
      The attributions are summed by default, but you can choose to average, etc.
    """

    merged_tokens = []
    merged_attribs = []

    current_word = ""
    current_attr_sum = 0.0

    for subword, attr in zip(tokens, attributions):
        # Remove leading 'Ġ' if present
        cleaned_sub = subword.lstrip("Ġ")

        # If the subword starts with "Ġ" or if current_word is empty,
        # we treat it as the start of a new word.
        # (For the very first token, current_word is empty, so we set it directly.)
        if subword.startswith("Ġ") or current_word == "":
            # If we already have a word pending, push it first
            if current_word != "":
                merged_tokens.append(current_word)
                merged_attribs.append(current_attr_sum)

            current_word = cleaned_sub
            current_attr_sum = attr
        else:
            # If subword doesn't start with 'Ġ', continue the same word
            current_word += cleaned_sub  # e.g. "Bl" + "end" -> "Blend"
            current_attr_sum += attr

    # After the loop, push the final word
    if current_word:
        merged_tokens.append(current_word)
        merged_attribs.append(current_attr_sum)

    return merged_tokens, merged_attribs


"""
def interpret_sentence(model, tokenizer, device, text):
    attributions_ig, delta_ig, pred_class, confidence, probs = get_explanations_ig(model, tokenizer, device, text)

    id2label = {0: 'bad', 1: 'average', 2: 'good', 3: 'excellent'}
    label_str = id2label[pred_class]

    attributions = attributions_ig.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    #tokens_list = tokenizer.tokenize(text)
    #clean_tokens_list = [token.replace("Ġ", "") for token in tokens_list]

    tokens_list = tokenizer.tokenize(text)
    # Suppose `attributions` is a 1D numpy array (or list) matching `tokens_list`
    merged_tokens, merged_attributions = merge_subwords(tokens_list, attributions)



    assert len(tokens_list) <= len(attributions), "Tokens must not be longer than attributions"

    visualization = viz.VisualizationDataRecord(word_attributions = merged_attributions, #attributions,
                                                pred_prob = confidence, #probs,
                                                pred_class = pred_class,
                                                true_class = None,
                                                attr_class = None,
                                                attr_score = sum(merged_attributions), #attributions.sum(),
                                                raw_input_ids = merged_tokens, #clean_tokens_list, #text,
                                                convergence_score = delta_ig)

    # Step B: Convert Captum HTML to a string
    # visualize_text expects an iterable of data records, so pass [record]
    out_html = viz.visualize_text([visualization])
    # out_html is an IPython.core.display.HTML object
    raw_html_str = out_html.data  # this is the underlying HTML string

    # Step C: Return the label, confidence, and the HTML string
    return label_str, confidence, raw_html_str
"""

#--------------

def format_word_importances_only(words, importances):
    """
    Returns a single <td> HTML cell containing the tokens
    colored according to their attributions.
    """
    if importances is None or len(importances) == 0:
        return "<td></td>"
    assert len(words) <= len(importances), (
        f"Found more tokens than attributions. "
        f"len(words)={len(words)} len(importances)={len(importances)}"
    )

    # Build HTML for token coloring
    token_html_list = []
    for word, imp in zip(words, importances[: len(words)]):
        color = _get_color(imp)
        token_html_list.append(
            f"<mark style='background-color:{color}; "
            "display:inline-block; line-height:1.75; "
            "margin:0 2px; border-radius:3px'>"
            f"{word}</mark>"
        )
    return "<td>" + "".join(token_html_list) + "</td>"

def _get_color(attr):
    """
    Internal helper to map attribution value to an RGBA color.
    This replicates Captum's logic for coloring tokens.
    """
    # You can tweak these constants if desired
    # Positive attributions = green, negative = red, magnitude scales opacity
    # If you don't have negative attributions, you can just do greens, etc.
    red, green, blue = (21, 177, 234) if attr > 0 else (234, 78, 21) #(255, 0, 0) if attr < 0 else (0, 255, 0)
    alpha = min(1.0, 0.2 + math.fabs(attr))  # scale transparency by magnitude
    return f"rgba({red},{green},{blue},{alpha})"

def visualize_text_only_word_importances(datarecords):
    """
    datarecords can be either a single VisualizationDataRecord or a list of them.
    Generates an HTML table with ONLY the 'Word Importance' column, removing
    'True Label', 'Predicted Label', 'Attribution Label', and 'Attribution Score'.
    """

    # If a single VisualizationDataRecord is passed, wrap in a list
    if not isinstance(datarecords, list):
        datarecords = [datarecords]

    # Table header: only one column (Word Importance)
    rows = ["<th>Word Importance</th>"]

    # For each VisualizationDataRecord, format one <tr>
    for dr in datarecords:
        # dr.word_attributions: np.array or list of attributions
        # dr.raw_input_ids: tokens
        word_importances_html = format_word_importances_only(
            dr.raw_input_ids, dr.word_attributions
        )
        row_html = f"<tr>{word_importances_html}</tr>"
        rows.append(row_html)

    table_html = "<table>" + "".join(rows) + "</table>"
    return HTML(table_html)
#----------------------------

def interpret_sentence(model, tokenizer, device, text):
    attributions_ig, delta_ig, pred_class, confidence, probs = get_explanations_ig(model, tokenizer, device, text)

    id2label = {0: 'bad', 1: 'average', 2: 'good', 3: 'excellent'}
    label_str = id2label[pred_class]

    attributions = attributions_ig.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    #tokens_list = tokenizer.tokenize(text)
    #clean_tokens_list = [token.replace("Ġ", "") for token in tokens_list]

    tokens_list = tokenizer.tokenize(text)
    # Suppose `attributions` is a 1D numpy array (or list) matching `tokens_list`
    merged_tokens, merged_attributions = merge_subwords(tokens_list, attributions)



    assert len(tokens_list) <= len(attributions), "Tokens must not be longer than attributions"

    visualization = viz.VisualizationDataRecord(word_attributions = merged_attributions, #attributions,
                                                pred_prob = confidence, #probs,
                                                pred_class = pred_class,
                                                true_class = None,
                                                attr_class = None,
                                                attr_score = np.mean(merged_attributions), #sum(merged_attributions), #attributions.sum(),
                                                raw_input_ids = merged_tokens, #clean_tokens_list, #text,
                                                convergence_score = delta_ig)

    # Step B: Convert Captum HTML to a string
    # visualize_text expects an iterable of data records, so pass [record]
    #out_html = viz.visualize_text([visualization])
    # out_html is an IPython.core.display.HTML object
    #raw_html_str = out_html.data  # this is the underlying HTML string
    html_obj = visualize_text_only_word_importances(visualization)
    # This is an IPython HTML object; get the raw string via `.data`
    raw_html_str = html_obj.data

    # Step C: Return the label, a percentage format confidence, and the HTML string
    return label_str, f'{100*confidence:.1f} %', raw_html_str

def create_gradio_interface():
    from functools import partial
    final_fn = partial(interpret_sentence, model, tokenizer, device)
    iface = gr.Interface(
        fn=final_fn,
        inputs=gr.Textbox(lines=5, label="Input Text"),
        outputs=[
            gr.Label(label="Predicted Class"),
            gr.Label(label="Confidence"),
            gr.HTML(label="Attribution Visualization"), #gr.Plot(label="Attribution Analysis")
        ],
        title="Advanced XAI Text Classification Explainer",
        description="""This application provides sophisticated explanations for text classification predictions using multiple XAI methods:
        1. Integrated Gradients: Shows how predictions change from baseline to input""",
        examples=[
            ["This wine is from Italy, White Blend variety. Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."],
            ["This wine is from US, Pinot Noir variety. Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew."],
            ["Baked plum, molasses, balsamic vinegar and cheesy oak aromas feed into a palate that's braced by a bolt of acidity. A compact set of saucy red-berry and plum flavors features tobacco and peppery accents, while the finish is mildly green in flavor, with respectable weight and balance."]
        ]
    )
    return iface

In [18]:
iface = create_gradio_interface()
iface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b1e7277358b56d4f90.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b1e7277358b56d4f90.gradio.live
